<img src=https://upload.wikimedia.org/wikipedia/commons/6/68/Logo_universidad_icesi.svg width=300>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sebastianb92/nlp-labs/blob/main/Session1/1-Spacy-Basics.ipynb)


# Maestría en Inteligencia Artificial  
## Procesamiento de Lenguaje natural
### Sesión 1 - Práctica

---


**Integrantes:**  
- Johan Sebastian Bonilla  
- Edwin Gómez  



# Caso práctico: Text Mining – FoodFast Delivery

FoodFast Delivery es una app de entrega de comida que recibe montones de comentarios de clientes y repartidores cada día.
El equipo de datos quiere automatizar el análisis de estos textos para poder:

- detectar las quejas más frecuentes

- encontrar segmentos de texto importantes

- extraer información clave (productos, ciudades, nombres)

- entender tendencias de los comentarios.

**Configurar entorno**

In [1]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

/tmp/ipython-input-2396000874.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [1]:
!test "$IN_COLAB" = "True" && wget -q https://raw.githubusercontent.com/sebastianb92/nlp-labs/main/requirements.txt -O requirements.txt && pip install -r requirements.txt
!python -m spacy download es_core_news_sm

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "/usr/local/lib/python3.12/dist-packages/spacy/__init__.py", line 16, in <module>
    from .cli.info import info  # noqa: F401
    ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/spacy/cli/__init__.py", line 12, in <module>
    from .convert import convert  # noqa: F401
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/spacy/cli/convert.py", line 13, in <module>
    from ..training.converters import (
  File "/usr/local/lib/python3.12/dist-packages/spacy/training/converters/__init__.py", line 2, in <module>
    from .conllu_to_docs import conllu_to_docs  # noqa: F401
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap

Reiniciamos el entorno

In [ ]:
import os
os.kill(os.getpid(), 9)

Para este caso trabajaremos con un documento en español

In [ ]:
# RUN THIS CELL to perform standard imports:
import spacy
nlp = spacy.load('es_core_news_sm')

**1. Cargamos documento**

In [ ]:
# Descargar el archivo correctamente
!wget -O comments.txt https://raw.githubusercontent.com/sebastianb92/nlp-labs/main/Session1/comments.txt

Teniendo en cuenta que el archivo es un documento tabulado, hacemos el cargue por lineas

In [ ]:
with open('comments.txt', 'r', encoding='utf-8') as file:
    comments = [line.strip() for line in file.readlines()]

docs = [nlp(comment) for comment in comments]

Mostramos los primeros 5 comentarios

In [ ]:
docs[:5]

**2. Cuantos comentarios (tokens) hay en el archivo?**

In [ ]:
len(docs)

El archivo cuenta con 52 comentarios

**3. Cuantas oraciones hay en el archivo?**

In [ ]:
for i, doc in enumerate(docs):
    print(f"Comentario {i}: {len(doc)} tokens, {len(list(doc.sents))} oraciones")


El documento cuenta con 53 oraciones, de los 52 comentarios, un comentario esta compuesto por 2 oraciones, el resto de los comentarios se componen de una sola oracion

**4. Por cada token en la oración anterior, imprime su text, POS tag, dep tag y lemma**

In [ ]:
import pandas as pd

rows = []
for token in docs[0]: ## Cambiar el comentario
    rows.append([token.text, token.pos_, token.dep_, token.lemma_])

pd.DataFrame(rows, columns=["Token","POS","Dep","Lemma"])

**5. Extracción de frases nominales y entidades**

Entidades = cosas específicas del mundo real (personas, lugares, empresas, fechas).

Noun chunks = frases nominales (cosas mencionadas, pero no necesariamente “famosas” o reconocibles como entidad).

In [ ]:
for doc in docs:
    print("Entidades:", [(ent.text, ent.label_) for ent in doc.ents])
    print("Noun chunks:", [chunk.text for chunk in doc.noun_chunks])

Se observa que muchos comentarios no necesariamente cuentan con una Entidad

**6. Construir un Matcher de quejas**

Con el fin de identificar los comentarios que referencian quejas, primero declaramos las frases que indican problemas con el servicio de domicilio

In [ ]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

patterns = [
    [{"LEMMA": "llegar"}, {"LOWER": "tarde"}],
    [{"LEMMA": "cobrar"}, {"LOWER": "doble"}],
    [{"LEMMA": "fallar"}],
    [{"LEMMA": "no"}, {"POS": "VERB"}]
]

matcher.add("QUEJAS", patterns)


Ahora buscamos los comentarios que coincidan con este patron de frases, definidas como posibles quejas

In [ ]:
def report_insights(docs):
    complaints = []
    for doc in docs:
        matches = matcher(doc)
        if matches:
            complaints.append(doc.text)
    return complaints

complaints = report_insights(docs)

for c in complaints:
    print(c)


📌 “¿Cuál es la ciudad con más quejas?”

📌 “¿Cuáles son los productos mencionados más frecuentemente con quejas?”

📌 “¿Qué porcentaje de comentarios son negativos?”

barras de frecuencia de palabras

wordcloud de quejas

Pie chart de tipos de quejas

**7. Identificar las ciudades con mas comentarios**

In [ ]:
from collections import Counter

# Supongamos que ya tienes tus comentarios
# comments = [...]  # lista de comentarios

# Contador de ciudades
cities_counter = Counter()

for comment in comments:
    doc = nlp(comment)
    for ent in doc.ents:
        if ent.label_ == "LOC" or ent.label_ == "GPE":  # ciudades o lugares
            cities_counter[ent.text] += 1

# Mostrar las ciudades con más menciones
most_common_cities = cities_counter.most_common(5)
print("Ciudades con más quejas:")
for city, count in most_common_cities:
    print(f"{city}: {count} quejas")


**8. Identificar el producto u objeto con mas comentarios**

In [ ]:
noun_counter = Counter()

for comment in comments:
    doc = nlp(comment)
    for token in doc:
        if token.pos_ == "NOUN":
            noun_counter[token.lemma_.lower()] += 1  # usamos lemma para agrupar variantes

# Top 10 productos mencionados
for noun, count in noun_counter.most_common(10):
    print(f"{noun}: {count}")


**9. Cual es el porecentaje de comentarios negativos?**

In [ ]:
neg_words = ["tarde", "fría", "demorado", "cancelado", "no funciona", "grosero", "perdió", "mal", "incorrecto", "derramada", "esperar", "pésimo"]


In [ ]:
neg_count = 0

for comment in comments:
    if any(word in comment.lower() for word in neg_words):
        neg_count += 1

total_comments = len(comments)
percent_negative = (neg_count / total_comments) * 100
print(f"Comentarios negativos: {percent_negative:.2f}%")


**10. Cuales son las palabras mas frecuentes en los comentarios?**

In [ ]:
# Contador de palabras
word_counter = Counter()

for comment in comments:
    doc = nlp(comment)
    for token in doc:
        # Solo palabras reales, sin puntuación ni espacios
        if token.is_alpha and not token.is_stop:
            word_counter[token.lemma_.lower()] += 1


In [ ]:
top_n = 20
most_common_words = word_counter.most_common(top_n)
words, counts = zip(*most_common_words)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(12,6))

norm = plt.Normalize(min(counts), max(counts))

# degradado más suave (evita blancos)
colors = plt.cm.Blues(0.3 + 0.7 * norm(counts))

plt.bar(words, counts, color=colors)
plt.xticks(rotation=45)
plt.title("Top 20 palabras más frecuentes en los comentarios")
plt.ylabel("Frecuencia")
plt.show()


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Concatenamos todos los comentarios en un solo string
text = " ".join(comments)

# Creamos el WordCloud
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color="white",
    colormap="Blues",       # colores tipo quejas
    max_words=100,         # máximo número de palabras a mostrar
    stopwords=None         # si quieres, spaCy stopwords
).generate(text)

# Mostrar el WordCloud
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("WordCloud de comentarios de quejas", fontsize=18)
plt.show()


In [ ]:
stopwords = nlp.Defaults.stop_words

wordcloud = WordCloud(
    width=800,
    height=400,
    background_color="white",
    colormap="Blues",
    max_words=100,
    stopwords=stopwords
).generate(text)


In [ ]:
# Mostrar el WordCloud
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("WordCloud de comentarios de quejas", fontsize=18)
plt.show()

**10. Identifique y grafique de forma agrupada, los tipos de quejas o inconformidades**

In [ ]:
from collections import Counter

categories_positive = {
    "Entrega rápida": ["rápido", "llegó rápido", "a tiempo", "en menos", "volando", "puntual"],
    "Comida deliciosa": ["delicioso", "rica", "sabrosa", "excelente", "muy buena", "perfecta", "caliente"],
    "Buen precio / Promos": ["barato", "económico", "promo", "descuento", "oferta", "precio justo"],
    "Repartidor amable": ["amable", "respetuoso", "cordial", "educado", "muy atento", "simpático"],
    "App fácil / buena": ["fácil", "intuitiva", "me encanta la app", "funciona bien", "muy práctica", "rápida la app"],
    "Buen servicio": ["excelente servicio", "muy buen servicio", "recomendado", "5 estrellas", "me encantó"]
}

category_counter_pos = Counter()

for comment in comments:
    comment_lower = comment.lower()
    matched = False

    for cat, keywords in categories_positive.items():
        for kw in keywords:
            if kw in comment_lower:
                category_counter_pos[cat] += 1
                matched = True
                break
        if matched:
            break

    if not matched:
        category_counter_pos["Otros"] += 1

print(category_counter_pos)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

labels = list(category_counter_pos.keys())
sizes = list(category_counter_pos.values())

colors = plt.cm.Blues(np.linspace(0.3, 0.9, len(labels)))

plt.figure(figsize=(8,8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors)
plt.title("Distribución de tipos de quejas")
plt.show()



In [ ]:
import re

categories = {
    "Retrasos": ["tarde", "demorado", "esperar", "retraso", "hora", "lento"],
    "Comida / Calidad": ["fría", "quemada", "aplastada", "mal olor", "derramada", "caliente", "deliciosa"],
    "Pago / Cobro": ["cobro", "tarjeta", "incorrecto", "doble"],
    "Entrega incompleta": ["faltaba", "incompleto", "perdió", "cancelado"],
    "Repartidor / Atención": ["grosero", "amable", "soporte", "no responde", "atento"],
}

# Contador de tipos de quejas
category_counter = Counter()

for comment in comments:
    comment_lower = comment.lower()
    matched = False
    for cat, keywords in categories.items():
        for kw in keywords:
            if kw in comment_lower:
                category_counter[cat] += 1
                matched = True
                break
        if matched:
            break
    if not matched:
        category_counter["Otros"] += 1

print(category_counter)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

labels = list(category_counter.keys())
sizes = list(category_counter.values())

colors = plt.cm.Blues(np.linspace(0.3, 0.9, len(labels)))

plt.figure(figsize=(8,8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors)
plt.title("Distribución de tipos de quejas")
plt.show()

